In [1]:
import xarray as xr
import os
data = xr.open_dataset(r"C:\Users\Pranjali\Downloads\Miroc_rcp_85_bcsd_2006_2100.nc")

In [2]:
t= data['Miroc_rcp_85_bcsd_2006_2100']

In [14]:
data

<xarray.Dataset>
Dimensions:                      (lat: 117, lon: 118, time: 34675)
Coordinates:
  * time                         (time) object 2006-01-01 00:00:00 ... 2100-12-31 00:00:00
  * lon                          (lon) float64 68.0 68.25 68.5 ... 97.0 97.25
  * lat                          (lat) float64 8.25 8.5 8.75 ... 37.0 37.25
Data variables:
    Miroc_rcp_85_bcsd_2006_2100  (time, lat, lon) float64 ...
Attributes:
    units:    mm/day
    name:     Miroc_rcp_85_bcsd_2006_2100
    rcp:      RCP_85

In [3]:
from rpy2.robjects.packages import importr
utils = importr("utils")
extremes = importr("extRemes")
ismev = importr("ismev")
eva = importr("eva")
gnFit = importr("gnFit")
from netCDF4 import Dataset
import xarray as xr
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib.offsetbox import AnchoredText
from netCDF4 import num2date, date2num
statis = importr('stats')
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

In [4]:
climate1 = t.sel(time = slice('2006-01-01','2036-12-31'))
climate2 = t.sel(time = slice('2037-01-01','2076-12-31'))

In [5]:
l = [climate1,climate2]

In [8]:
#l1=[]
#for i in l:
climate = climate1.where(climate1>=0)
ppt = climate
maxima1 = ppt.groupby('time.year').max('time')
rlv1=[]
rlv301=[]
paramlist1=[]

lon1=[]
lat1=[]
for i in range(117):
    for j in range(118):
        maxima11 = maxima1.sel(lat=data.lat[i],lon=data.lon[j])
        params1 = {}    
        if(maxima11.values[1]>0):
            #RL2 = []
            block_maxima=maxima11.values[~np.isnan(maxima11.values)]
            try:
                gev_fit =  eva.gevrFit(block_maxima)
            except:
                pass
            MLE_est = list(gev_fit.rx2('par.ests'))

            #print(block_maxima)


            params1['location'] = MLE_est[0]

            params1['scale'] =  MLE_est[1] 

            params1['shape'] = MLE_est[2]

            if MLE_est[1]<0:
                MLE_est[1] = 0.01

            y = eva.rgevr(10000, 1, loc = MLE_est[0], scale = MLE_est[1], shape = MLE_est[2])

            sig_test = statis.ks_test(block_maxima, y)

            params1['p_val'] = sig_test.rx2('p.value')[0]
            paramlist1.append(params1)

    #print(params)
  


            if params1['p_val']>0.05:

                temp = eva.gevrRl(gev_fit, 100, method = "delta")
                temp1 = eva.gevrRl(gev_fit, 30, method = "delta")
                RL1 = temp.rx2('Estimate')[0]
                RL301 = temp1.rx2('Estimate')[0]
#n['RLV2'][i][j] = RL2

            else:
                params1['p_val'] = np.nan
                params1['location'] = np.nan
                params1['scale'] = np.nan
                params1['shape'] = np.nan
            #print(RL2,climate.lat.values[i],climate.lon.values[j])
            rlv1.append(RL1)
            rlv301.append(RL301)
            lat1.append(data.lat.values[i])
            lon1.append(data.lon.values[j])
#n['RLV2'][i][j] = np.nan
#print(RL2)
        else:
            params1['p_val'] = np.nan
            params1['location'] = np.nan
            params1['scale'] = np.nan
            params1['shape'] = np.nan
df1 = pd.DataFrame({'rlv5':rlv1,'rlv305':rlv301,'lat1':lat1,'lon1':lon1})
    #l1.append(df1)

In [9]:
ds1= df1.set_index(['lat1','lon1']).to_xarray()
ds1.to_netcdf(path=r'D:\All project data\MirocESM1-2006-2036-30100.nc')